Your Name:  Gregory Roberts

Submission Instruction:
* Instruction: make a copy of this CoLab file and shared it with me (Yifeng.Zhu@maine.edu).
* Deadline: Midnight, Sunday, March 6
* Reference code: https://colab.research.google.com/drive/1mBMoYnX9BuMGeymVbmcx-pC-qPR9lARH?usp=sharing



Write short python programs to answer the following questions

In [ ]:
import numpy as np
import pandas as pd
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
import statistics

# Question 1. Paired Comparison

* We are trying to evaluate the performance impact of a new hard drive.
* We measure the response time on the old computer systems, and also the new systems with the new harddrive.
* Can we say that the harddrive improves the performance with a confidence of 95%?

In [ ]:
old = [6.37, 5.69, 5.58, 5.27, 5.11, 4.89, 4.70, 3.53]  # Runnimg time on old systems using 8 applications
new = [4.52, 5.44, 4.70, 3.81, 4.06, 3.22, 2.96, 3.20]  # Runnimg time on new systems using the same 8 applications

In [ ]:
array1 = np.array(old)
array2 = np.array(new)

data = np.subtract(array1, array2)

print(data) # Difference in run time between old and new systems


[1.85 0.25 0.88 1.46 1.05 1.67 1.74 0.33]


In [ ]:
alpha = 0.95
ci = alpha + (1-alpha)/2
df = len(data)-1
# ppf = Percent point function (inverse of cdf)
c_lb = np.mean(data) - stats.t.ppf(ci, df)*stats.sem(data) # Lower bound
c_ub = np.mean(data) + stats.t.ppf(ci, df)*stats.sem(data) # Upper bound
print(c_lb, c_ub)

0.6278643124352861 1.679635687564714


**The confidence interval does not include zero. Thus, we can say with 95% confidence that the harddrives performance improved.**

---



# Question 2. Unpaired Comparison

* We are comparing the response time of two I/O systems (A and B) using the same benchmark. 
* Can we be 95% sure that the mean response time of System A is higher than System B? (i.e., Can we be 95% sure that System B is faster?)

In [ ]:
a = [186, 181, 176, 149, 184, 190, 158, 139, 175, 148, 152, 111, 141, 153, 190, 157, 131, 149, 135, 132] # Response time of System A
b = [129, 132, 102, 106, 94, 102, 87, 99, 170, 113, 135, 142, 86, 143, 152, 146, 144] # Response time of System B
print(len(a), ' tests on System A')
print(len(b), ' tests on System B')

20  tests on System A
17  tests on System B


In [ ]:
def degreesOfFreedom(X, Y):
  s1 = (statistics.stdev(X)**2)
  s2 = (statistics.stdev(Y)**2)
  a, b = s1/len(X), s2/len(Y)
  df = (a + b)**2 / ((a)**2 / (len(X) - 1) + (b)**2 / (len(Y) - 1))
  return(df)


In [ ]:
# function for calculating the t-test for two independent samples
def independent_ttest(data1, data2, alpha):

	# calculate means
  mean1, mean2 = np.mean(data1), np.mean(data2)

	# calculate standard errors
  se1, se2 = stats.sem(data1), stats.sem(data2)

	# standard error on the difference between the samples
  sed = math.sqrt(se1**2.0 + se2**2.0)

	# calculate the t statistic
  t_stat = (mean1 - mean2) / sed

	# degrees of freedom
	# df = len(data1) + len(data2) - 2
  df = degreesOfFreedom(data1, data2)

	# calculate the critical value
  cv = stats.t.ppf(1.0 - alpha, df)

	# calculate the p-value
  p = (1.0 - stats.t.cdf(abs(t_stat), df)) * 2.0
	
	# return everything
  return t_stat, df, cv, p

In [ ]:
degreesOfFreedom(a, b)

32.39414850154182

In [ ]:
# calculate the t test
alpha = 0.05
t_stat, df, cv, p = independent_ttest(a, b, alpha)
print('t=%.3f, df=%d, cv=%.3f, p=%.3f' % (t_stat, df, cv, p))
mean1, mean2 = np.mean(a), np.mean(b)
print('mean1=%d, mean2=%d' % (mean1, mean2))

# interpret via critical value
if abs(t_stat) <= cv:
	print('Accept null hypothesis that the means are equal.')
else:
	print('Reject the null hypothesis that the means are equal.')
 
# interpret via p-value
if p > alpha:
	print('Accept null hypothesis that the means are equal.')
else:
	print('Reject the null hypothesis that the means are equal.')

t=4.303, df=32, cv=1.693, p=0.000
mean1=156, mean2=122
Reject the null hypothesis that the means are equal.
Reject the null hypothesis that the means are equal.


**In both interpretations Critical Value and p-Value, we reject the null hypothesis.  As the means are considered equal for between both sets of responses.**

---



# Question 3. One-factor (one-way) ANOVA

Suppose that we evaluating three different compilers, including R, V, and Z. In all experiments, we use a compiler to compile the same program into an execuatble and measure the running time on the same computer. For each compiler, we measure five times, and each time a different optimization option is given to the compiler.  The running measurements are given below. 

In [ ]:
# Measurements of running time for three different compilers R, V, Z
R = [144, 120, 176, 288, 144]
V = [101, 144, 211, 288, 72]
Z = [130, 180, 141, 374, 302]

In [ ]:
np.mean(R), np.mean(V), np.mean(Z)

(174.4, 163.2, 225.4)

Is the observed difference in the mean mostly due to measurement errors or due to the difference among compilers? 

Part A: use the function stats.f_oneway to find the answer to the question

In [ ]:
F, p = stats.f_oneway(R, V, Z)
print(F, p)

0.6989101914688891 0.5162767522588427


**The p value obtained from ANOVA analysis is significant (p > 0.05), and therefore, we conclude that there are no significant differences among treatments.
F value is closely related to p value and F value indicates that the p value is not significant.**


---



Part B: find the answer to the question using step-by-step calulations similar to the lecture slides and [google code lab example](https://colab.research.google.com/drive/1Ueoi5Ynp5UAcYD-23Ue4goYXtiLTpUQy?usp=sharing) (you cannot use the function stats.f_oneway).

In [ ]:
all_times = R + V + Z
system_names = (['R'] * len(R)) +  (['V'] * len(V)) +  (['Z'] * len(Z))
data = pd.DataFrame({'compiler': system_names, 'running time': all_times})
data

,compiler,running time
0,R,144
1,R,120
2,R,176
3,R,288
4,R,144
5,V,101
6,V,144
7,V,211
8,V,288
9,V,72


In [ ]:
data.groupby('compiler').mean()

,running time
compiler,
R,174.4
V,163.2
Z,225.4


In [ ]:
overall_mean = data['running time'].mean()
overall_mean

187.66666666666666

In [ ]:
# compute Sum of Squares Total
data['overall_mean'] = overall_mean
ss_total = sum((data['running time'] - data['overall_mean'])**2)
ss_total

105357.33333333333

In [ ]:
# compute group means
group_means = data.groupby('compiler').mean()
group_means = group_means.rename(columns = {'running time': 'group_mean'})
group_means

,group_mean,overall_mean
compiler,,
R,174.4,187.666667
V,163.2,187.666667
Z,225.4,187.666667


In [ ]:
# add group means and overall mean to the original data frame
data = data.drop('overall_mean', axis=1)
data = data.merge(group_means, left_on='compiler', right_index=True, copy=False)
data

,compiler,running time,group_mean,overall_mean
0,R,144,174.4,187.666667
1,R,120,174.4,187.666667
2,R,176,174.4,187.666667
3,R,288,174.4,187.666667
4,R,144,174.4,187.666667
5,V,101,163.2,187.666667
6,V,144,163.2,187.666667
7,V,211,163.2,187.666667
8,V,288,163.2,187.666667
9,V,72,163.2,187.666667


In [ ]:
# compute Sum of Squares Residual
# Within-group Variability / unexplained variance
# Denoted as SSE in the lecture slide
ss_residual = sum((data['running time'] - data['group_mean'])**2)
ss_residual

94365.20000000001

In [ ]:
# compute Sum of Squares Model
# Explained Variance or Between-group Variability 
# Denoted as SSA in the lecture slides
ss_explained = sum((data['overall_mean'] - data['group_mean'])**2)
ss_explained

10992.133333333339

In [ ]:
n_groups = len(set(data['compiler']))
n_obs = data.shape[0]
df_residual = n_obs - n_groups
df_residual

12

In [ ]:
# compute Mean Square Residual
# Mean within-group variability / unexplained variance
ms_residual = ss_residual / df_residual
ms_residual

7863.766666666667

In [ ]:
df_explained = n_groups - 1
df_explained

2

In [ ]:
# compute Mean Square Explained
ms_explained = ss_explained / df_explained
ms_explained

5496.066666666669

In [ ]:
# compute F-Value
f = ms_explained / ms_residual
f

0.6989101914688892

In [ ]:
# compute p-value
p_value = 1 - stats.f.cdf(f, df_explained, df_residual)
p_value

0.5162767522588427

**The function stats.f_oneway produced the following for F and p value:**

***0.6989101914688891 0.5162767522588427***

**Which is almost the same value for F up to the 15th significant digit.**

**So the p value obtained from ANOVA analysis is significant (p > 0.05), and therefore, we conclude that there are no significant differences among treatments. F value is closely related to p value and F value indicates that the p value is not significant.**


---




